In [1]:
1+1

2

In [2]:
import pandas as pd 

huhu = pd.read_parquet(path="output/delta_chunks_to_index.parquet")

In [3]:
huhu.shape

(114797, 6)

In [4]:
hihi = pd.read_parquet(path="output/1_manual_law_strict.parquet")

In [5]:
hihi.shape

(2084, 7)

In [6]:
hihi.head()

,chunk_id,doc_title,doc_category,doc_url,vector_text,display_text,crawled_at
0,law_luat_hon_nhan_va_gia_dinh.txt_0,Luật Hôn nhân và Gia đình 2014,Pháp luật,local/luat_hon_nhan_va_gia_dinh.txt,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,2025-12-11T10:46:46.879494
1,law_luat_hon_nhan_va_gia_dinh.txt_1,Luật Hôn nhân và Gia đình 2014,Pháp luật,local/luat_hon_nhan_va_gia_dinh.txt,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,2025-12-11T10:46:46.879494
2,law_luat_hon_nhan_va_gia_dinh.txt_2,Luật Hôn nhân và Gia đình 2014,Pháp luật,local/luat_hon_nhan_va_gia_dinh.txt,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,2025-12-11T10:46:46.879494
3,law_luat_hon_nhan_va_gia_dinh.txt_3,Luật Hôn nhân và Gia đình 2014,Pháp luật,local/luat_hon_nhan_va_gia_dinh.txt,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,2025-12-11T10:46:46.879494
4,law_luat_hon_nhan_va_gia_dinh.txt_4,Luật Hôn nhân và Gia đình 2014,Pháp luật,local/luat_hon_nhan_va_gia_dinh.txt,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,Văn bản: Luật Hôn nhân và Gia đình 2014\nChươn...,2025-12-11T10:46:46.879494


In [7]:
huhu.head()

,chunk_id,doc_title,doc_category,vector_text,display_text,doc_url
0,Anh hùng dân tộc Việt Nam_0,Anh hùng dân tộc Việt Nam,Nhân vật lịch sử Việt Nam,Lĩnh vực: Nhân vật lịch sử Việt Nam. Chủ đề: A...,Anh hùng dân tộc Việt Nam là thuật ngữ chỉ nhữ...,https://vi.wikipedia.org/wiki/Anh_h%C3%B9ng_d%...
1,Anh hùng dân tộc Việt Nam_1,Anh hùng dân tộc Việt Nam,Nhân vật lịch sử Việt Nam,Lĩnh vực: Nhân vật lịch sử Việt Nam. Chủ đề: A...,"Đinh Tiên Hoàng, tức Đinh Bộ Lĩnh: người đánh ...",https://vi.wikipedia.org/wiki/Anh_h%C3%B9ng_d%...
2,Anh hùng dân tộc Việt Nam_2,Anh hùng dân tộc Việt Nam,Nhân vật lịch sử Việt Nam,Lĩnh vực: Nhân vật lịch sử Việt Nam. Chủ đề: A...,Tiêu chuẩn\n\n14 vị Anh hùng dân tộc Việt Nam ...,https://vi.wikipedia.org/wiki/Anh_h%C3%B9ng_d%...
3,An Dương Vương_0,An Dương Vương,Triều đại Việt Nam,Lĩnh vực: Triều đại Việt Nam. Chủ đề: An Dương...,"An Dương Vương (chữ Hán: 安陽王), tên thật là Thụ...",https://vi.wikipedia.org/wiki/An_D%C6%B0%C6%A1...
4,Nhà nước Việt Nam_0,Nhà nước Việt Nam,Nhà nước Việt Nam,Lĩnh vực: Nhà nước Việt Nam. Chủ đề: Nhà nước ...,Nhà nước Cộng hòa xã hội chủ nghĩa Việt Nam là...,https://vi.wikipedia.org/wiki/Nh%C3%A0_n%C6%B0...
